In [ ]:
import pickle
from GAMERNet.rnet.networks.reaction_network import ReactionNetwork
from graph_tool.topology import all_paths, shortest_path
from graph_tool import Graph
import pprint as pp

In [ ]:
with open('../scripts/C2_Cu100/rxn_net.pkl', 'rb') as pickle_file:
    content = pickle.load(pickle_file)  # dict of elementary reactions

rxn_net = ReactionNetwork().from_dict(content)

rxn_net_graph = rxn_net.gen_graph()

In [ ]:
# Remove edges that contain the following species (avoiding getting unwanted paths)
edge_rm_lst = []
for edge in rxn_net_graph.edges():
    ignored_species = set(['000000*', '010101*', '011101*', '001101*'])
    if ignored_species.intersection(set(edge)):
        edge_rm_lst.append(edge)
rxn_net_graph.remove_edges_from(edge_rm_lst)
print('Edges removed: ', len(edge_rm_lst))

In [ ]:
vertex_map = {name: index for index, name in enumerate(set([v for edge in rxn_net_graph.edges() for v in edge]))}
vertex_map

In [ ]:
integer_edge_list = [[vertex_map[edge[0]], vertex_map[edge[1]]] for edge in rxn_net_graph.edges()]
integer_edge_list.extend([[vertex_map[edge[1]], vertex_map[edge[0]]] for edge in rxn_net_graph.edges()])

In [ ]:
graph_tool_graph = Graph(directed=True)
graph_tool_graph.add_edge_list(integer_edge_list)

In [ ]:
starting_point = vertex_map['102101g']
ending_point = vertex_map['260101g']


In [ ]:
all_pths = all_paths(graph_tool_graph, starting_point, ending_point,cutoff=10)

In [ ]:
all_paths_lst = []
for pth in all_pths:
    # Assigning the path to the graph
    path = []
    for node in pth:
        path.append(list(vertex_map.keys())[list(vertex_map.values()).index(node)])
    all_paths_lst.append(path)

all_paths_lst
pp.pprint(all_paths_lst)

# Printing the number of paths
print("Number of paths: ", len(all_paths_lst))

In [ ]:
shortest_pth = shortest_path(graph_tool_graph, starting_point, ending_point)

shortest_pth_labels = [list(vertex_map.keys())[list(vertex_map.values()).index(node)] for node in shortest_pth[0]]
shortest_pth_labels